# Embedded AI&ML Test Cases

## MNIST Handwritten Digits

In [ ]:
import tensorflow as tf
print("Using TensorFlow version", tf.__version__)

from tensorflow import keras
print("Using Keras version     ", keras.__version__)
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping

from sklearn import metrics
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"

## Download the MNIST dataset

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

width = X_train.shape[2]
height = X_train.shape[1]

## Start Processing the Images

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils

def decode_one_hot(y):
    y_classes = [np.argmax(yi, axis=None, out=None) for yi in y]
    return y_classes

# Use one-hot coding to code the labels into binary.  The neural net needs this.
y_train = y_train.astype(int)
y_train = np_utils.to_categorical(y_train) # binarize the integer class ID's

y_test = y_test.astype(int)
y_test = np_utils.to_categorical(y_test) # binarize the integer class ID's

# Scale the X parameters for processing by the neural net
X_train = X_train.reshape(-1, 1)
X_train = MinMaxScaler(feature_range=(-1, 1)).fit_transform(X_train)

X_test = X_test.reshape(-1, 1)
X_test = MinMaxScaler(feature_range=(-1, 1)).fit_transform(X_test)

# Reshape X correctly to (images, width, height)
X_train = X_train.reshape(-1, width, height)
X_test = X_test.reshape(-1, width, height)

print ("X_train shape:", X_train.shape)
print ("y_train shape:", y_train.shape)

print ("X_test shape:", X_test.shape)
print ("y_test  shape:", y_test.shape)

# Split half of the test set into a validation set
split = int(X_test.shape[0] / 2)

X_val = X_test[split:]
y_val = y_test[split:]

X_test = X_test[0:split]
y_test = y_test[0:split]

# Reshape X_train, X_test, X_val to (images, width, height)
X_train = X_train.reshape(X_train.shape[0], width, height, 1)
X_test = X_test.reshape(X_test.shape[0], width, height, 1)
X_val = X_val.reshape(X_val.shape[0], width, height, 1)

# Input shape for the first layer of the Neural Net
input_shape = (width, height, 1)

print("X_train shape  X_test shape  X_val shape")
print(X_train.shape, X_test.shape, X_val.shape)
print("y_train shape  y_test shape  y_val shape")
print(y_train.shape, y_test.shape, y_val.shape)

## Display the images

Here's how we can display the images if that is interesting.  The example below displays the first 100ish images in the dataset.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

def display_image(array):
    %pylab inline    
    img = Image.fromarray(array)
    imgplot = plt.imshow(img)
    plt.show()

# Convert images to int8 and scale for plotting by imshow
X_plot = (X_train * 255).astype(np.int8)
X_plot = X_plot.reshape(-1, width, height)
print ("X_plot Shape:", X_plot.shape)

dec_y = decode_one_hot(y_train)
for i,d in enumerate(X_plot):
    display_image(d)
    print(dec_y[i])
    if i == 100:
        break

## Define and load the model

In [ ]:
learning_rate = .001

## Define the Model
model = Sequential()
model.add(Conv2D(10, kernel_size=(5,5), strides=(1,1), padding='same', activation="tanh", input_shape=input_shape))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(32, activation="tanh"))
model.add(Dropout(0.2))
model.add(Dense(10, activation="softmax"))


optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['categorical_accuracy'])

model.summary()

## Train the Neural Network

In [ ]:
epochs = 20
batch_size = 20

## Fit the Model
hist = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=False,
          validation_data=(X_test, y_test))

## Score the Results

In [ ]:
## Evaluate the Model

score = model.evaluate(X_test, y_test)
score_val = model.evaluate(X_val, y_val)

yhat = np.argmax(model.predict(X_test), axis=-1)
yhat_val = np.argmax(model.predict(X_val), axis=-1)

y_test1 = decode_one_hot(y_test)
y_val1 = decode_one_hot(y_val)

print()
print(metrics.classification_report(y_test1, yhat))
print(metrics.confusion_matrix(y_test1,yhat))
print("Testing Loss:", score[0])
print("Testing Accuracy:", score[1])

print()
print(metrics.classification_report(y_val1, yhat_val))
print(metrics.confusion_matrix(y_val1, yhat_val))
print("Validation Loss:", score_val[0])
print("Validation Accuracy:", score_val[1])


## Plot the accuracy vs. validation accuracy

epoch_list = list(range(1, len(hist.history['categorical_accuracy']) + 1))
plt.plot(epoch_list, hist.history['categorical_accuracy'], epoch_list, hist.history['val_categorical_accuracy'])
plt.legend(("Training Accuracy", "Validation Accuracy"))
plt.show()
plt.plot(epoch_list, hist.history['loss'], epoch_list, hist.history['val_loss'])
plt.legend(("Training Loss", "Validation Loss"))
plt.show()